# Create a tasking order

## Step 1. Set up the notebook

### 1. Install dependencies

In [ ]:
!pip install up42-py --upgrade -q

import up42, geojson, pathlib
import pandas as pd
from dataclasses import asdict

### 2. Authenticate

Create a `credentials.json` file in a directory named `.up42` under your home directory by running the cell below. The path to the file will be `~/.up42/credentials.json`.

In [ ]:
# Define the credentials file path if it doesn't exist
credentials_file_path = pathlib.Path.home().joinpath(".up42/credentials.json")
credentials_file_path.parent.mkdir(parents=True, exist_ok=True)
credentials_file_path.touch(exist_ok=True)

# Print the path to the file
print(f"Credentials file is located at: {credentials_file_path}")

1. Click the link above to the created file and paste the following code:
    ```
    {
        "username": "<your-email-address>",
        "password": "<your-password>"
    }
    ```
2. Retrieve the email address and password used for logging into the console. Use them as values for `username` and `password`.
3. Save the `credentials.json` file.
4. Check that the authentication was successful as follows:

In [ ]:
up42.authenticate(cfg_file=credentials_file_path)

## Step 2. Get access

### 1. Select a tasking collection

Choose a tasking collection and get a data product associated with it.

In [ ]:
# Fetch all collections
tasking_collections = list(up42.ProductGlossary.get_collections(collection_type=up42.CollectionType.TASKING))

# Define output
def print_product_info(product):
    info = asdict(product)
    description = info.get('description', '').replace('\n', '\\n')
    print(f"Name: {info.get('name', '')}")
    print(f"Title: {info.get('title', '')}")
    print(f"Description: {description}")
    print(f"ID: {info.get('id', '')}")
    print(f"EULA ID: {info.get('eula_id', '')}\n")

# Print information for all data products
for collection in tasking_collections:
    for product in collection.data_products:
        print_product_info(product)

Choose a data product.

In [ ]:
# The following value is the data product name for Maxar 30 cm
data_product_name = "maxar-30cm-tasking"

# Retrieve the associated data product ID
data_product_id = next((p.id for c in tasking_collections for p in c.data_products if p.name == data_product_name), None)

### 2. Request access

If you want to order the chosen collection for the first time, you need to [request access to it](https://docs.up42.com/getting-started/restrictions#request-access).

An email from the Customer Success team usually takes up to 3 days. You can review your access request status on the [Access requests](https://console.up42.com/settings/access) page.

### 3. Accept the EULA

If you want to order the chosen collection for the first time, you need to accept its [end-user license agreement (EULA)](https://docs.up42.com/getting-started/account/eulas).

## Step 3. Create an order

### 1. Fill out an order form

Get detailed information about the [parameters](https://docs.up42.com/data/tasking/order/form) needed to create an order for the chosen data product.

In [ ]:
# Create a dictionary of all data product names and their schemas
product_schemas = {
    product.name: product.schema
    for collection in tasking_collections
    for product in collection.data_products
}

# Look up the schema for the selected data product
schema = product_schemas.get(data_product_name)
schema

Before creating an order, find out how order parameters affect the [timeframe of your tasking order](https://docs.up42.com/data/tasking/timeframes/acquisition).

Specify the geometry and use the required request body schema format for the chosen data product.

In [ ]:
aoi_geometry = {
    "type": "Polygon",
    "coordinates": (
        (
            (13.306810340794902, 52.468797326826575),
            (13.306810340794902, 52.56290967317342),
            (13.447469659205202, 52.56290967317342),
            (13.447469659205202, 52.468797326826575),
            (13.306810340794902, 52.468797326826575),
        ),
    ),
}

features = geojson.FeatureCollection(features=[geojson.Feature(geometry=aoi_geometry)])

order_template = up42.BatchOrderTemplate(
    data_product_id=data_product_id,
    display_name=f"My tasking order for Maxar 30 cm",
    features=features,
    params={
        "acquisitionStart": "2026-01-17T22:00:00.000Z",
        "acquisitionEnd": "2026-05-17T22:00:00.000Z",
        "acquisitionMode": "mono",
        "maxCloudCover": 25,
        "maxIncidenceAngle": 10,
        "geometricProcessing": "georectified",
        "pixelCoding": "16bit",
        "spectralBands": "bundle_8_band",
        "priority": "standard",
        "radiometricProcessing": "toa",
    },
    tags=["project-maxar", "optical"]
)

Check the parameters by printing them.

In [ ]:
print("\nOrder parameters:")
for key, value in order_template.params.items():
    print(f"    - {key}: {value}")

### 2. Get a cost estimate

Get a cost estimation before creating an order.

In [ ]:
# Estimate the cost
estimate = order_template.estimate

# Define output
print(f"Total credits: {estimate.credits}")
print(f"Total size: {estimate.size} {estimate.unit}")
print("Geometry items:")
for item in estimate.items:
    print(f"    - Index: {item.index}")
    print(f"      Credits: {item.credits}")
    print(f"      Size: {item.size} {item.unit}")

The response returns the overall credit amount that will be deducted from your credit balance if you decide to proceed with the ordering.

### 3. Create an order

Create an order with the requested parameters.

In [ ]:
# Create orders defined in the template
# If your order_template has several separate AOIs that result in several orders, this will create all of them
order_references = order_template.place()
order = order_references[0].order
order

If you've defined an AOI with multiple geometries, each geometry will create a separate order. Order names will be suffixed with incrementing numbers: `Order 1`, `Order 2`, etc.

## Step 4. Review feasibility study options

### 1. Review options

Depending on the collection and parameter combination, your order might or might not require a [feasibility study](https://docs.up42.com/data/tasking/timeframes/feasibility).

* **If a feasibility study isn’t required**

    The order is considered feasible within the specified parameters and will move to the activation request stage.

* **If a feasibility study is required**

    After an order is placed, the Operations team will conduct a feasibility study. They will evaluate the tasking parameters with the provider, and then will present the following assessments:

    * The order is possible with the given parameters.
    * The order requires modifications with suitable options proposed.

You will receive an email notifying you when your order feasibility has been assessed. Using your order ID, review the proposed feasibility options.

In [ ]:
tasking = up42.initialize_tasking()
tasking.get_feasibility(order_id=order.id)

If none of the options are suitable, [contact](https://up42.com/company/contact-ordering) the Operations team.

### 2. Accept an option

You can't modify your order after accepting an option.

In [ ]:
tasking.choose_feasibility(
    feasibility_id="3bf00b63-7188-4b04-aa27-d78e4cd32c01",
    accepted_option_id="033b4a5a-c492-4eba-915e-2000a0a84049"
)

## Step 5. Activate an order

After accepting a feasibility study option, you will receive a price quote for your order. Using your order ID, review the received quotation.

In [ ]:
quotations = list(up42.Quotation.all(order_id=order_id))

# First, check if any quotations were found at all
if not quotations:
    print("No quotations found for this order.")
else:
    count = len(quotations)
    print(f"Found {count} quotation(s)")
    print("----------------------------")

    # Loop through all found quotations and print their details
    for i, quotation in enumerate(quotations):
        print(f"Quotation ID: {quotation.id}")
        print(f"Order ID:     {quotation.order_id}")
        print(f"Status:       {quotation.decision}")
        print(f"Price:        {quotation.credits_price:,} credits")

        if i < count - 1:
            print("---")

To activate your order, accept the quotation. Once you accept the tasking quotation, the credits for that order will be deducted. The transaction can’t be reversed.

In [ ]:
# Select a specific quotation by its ID
quotation_id = "ea36dee9-fed6-457e-8400-2c20ebd30f44"
selected_quotation = next((q for q in quotations if q.id == quotation_id), None)

selected_quotation.accept()
selected_quotation.save()

# Or, to reject:
# selected_quotation.reject()
# selected_quotation.save()

## Step 6. Monitor orders

Check the [status](https://docs.up42.com/developers/api-tasking#order-statuses) of your order.

In [ ]:
order.status

## Step 7. Download assets

The following statuses mean you can download assets from storage:
- `BEING_FULFILLED`: Some order assets might have been delivered.
- `FULFILLED`: All order assets have been delivered.